In [1]:
pip install -q langchain langchain-community langchain_huggingface chromadb

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.2 which is incompatible.
jupyterlab 4.2.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
kfp 2.5.0 requires kubernetes<27,>=8.0.0, but you have kubernetes 31.0.0 which is incompatible.
kfp 2.5.0 requires requests-toolbelt<1,>=0.8.0, but you have requests-toolbelt 1.0.0 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.

## 1. Use langchain RAG

In [2]:
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [5]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "your_API"

In [ ]:
# os.environ.get("HUGGINGFACEHUB_API_TOKEN")

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub

# set Korean embedding and llm odel
hf_embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")

hf_llm = HuggingFaceHub(
    repo_id="skt/kogpt2-base-v2",
    model_kwargs={"task": "text-generation"} ## question-answering tasK X. text-generation
)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
import requests
from langchain.schema import Document
from bs4 import BeautifulSoup

# for Wikipedia documents (EN, KO)

# from langchain_community.document_loaders import WikipediaLoader

# By default, English documents (https://en.wikipedia.org))
# def load_Wiki_docs(query):
#     loader = WikipediaLoader(query=query, load_max_docs=1) # need !pip install wikipedia
#     documents = loader.load()
    
#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=1000,
#         chunk_overlap=200
#     )
#     splits = text_splitter.split_documents(documents)
    
#     return splits


# For Korean query, get results from Korean wikipedia website and crawl and parse results
def load_Korean_wiki_docs(topic):
    url = f"https://ko.wikipedia.org/wiki/{topic}"
    
    response = requests.get(url)
    response.raise_for_status()  # raise Exception when error occurs

    # HTML parsing and extract body contents
    soup = BeautifulSoup(response.text, 'html.parser')
    content = soup.find('div', {'class': 'mw-parser-output'})  # find div including body contents 
    
    # Extract contents
    paragraphs = content.find_all('p')
    text = "\n".join([p.get_text() for p in paragraphs])  # concat all context in <p> tags 
 
    # convert to Document object (required for LangChain)
    documents = [Document(page_content=text, metadata={"source": url})]
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    splits = text_splitter.split_documents(documents)
    
    return splits

In [9]:
def create_vectorstore(splits): 
    vectorstore = Chroma.from_documents(documents=splits, embedding=hf_embeddings)
    return vectorstore

In [18]:
topic = "흑백요리사"
# Load wikipedia documents for this topic
splits = load_Korean_wiki_docs(topic) 
# Create vectorstore with this fetched docs
vectorstore = create_vectorstore(splits)

In [19]:
len(splits)

1

In [20]:
splits

[Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1]')]

In [21]:
def create_rag_chain(vectorstore):
    prompt_template = """문맥을 참고하여 질문에 정확하고 간결하게 답하십시오.
    문맥: {context}
    질문: {question}
    답변:"""
    
    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    chain_type_kwargs = {"prompt": PROMPT}

    # Make context shorter
    # def short_context(context, max_length=300):
    #     return context[:max_length] if len(context) > max_length else context
    
    # class ShortContextRetriever(BaseRetriever):
    #     def __init__(self, retriever):
    #         super().__init__()
    #         self._retriever = retriever
        
    #     def get_relevant_documents(self, query):
    #         docs = self._retriever.get_relevant_documents(query)
    #         for doc in docs:
    #             doc.page_content = short_context(doc.page_content)
    #         return docs
    
    # retriever = ShortContextRetriever(vectorstore.as_retriever())

    qa_chain = RetrievalQA.from_chain_type(
        llm=hf_llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs=chain_type_kwargs,
        return_source_documents=True
    )
    
    return qa_chain

In [22]:
# create langchang RAG chain
qa_chain = create_rag_chain(vectorstore)

In [24]:
question = "심사위원?"

# result = qa_chain({"query": question})
result = qa_chain.invoke({"query": question})

print ("결과:")
print(result["result"])

print("출처:")
for doc in result["source_documents"]:
    print(doc.page_content)
    print("---")

결과:
문맥을 참고하여 질문에 정확하고 간결하게 답하십시오.
    문맥: 15년 11월 2일, 아모레퍼시픽이 업계와 면접 당사자에 따르면, 한 면접관은 영업관리직무 2차 면접 시험장에서 "얼마 전 박근혜 대통령님이 국회에서 시정연설을 하며 강한 의지를 표하신 국정교과서에 대하여서 어떤 생각이냐"라는 질문을 하였다. 응시자가 "국정교과서가 사실상 바람직한 결정이라고 할 수 없다" 며 의견을 피력하자, 면접관이 "그래서 국정교과서 찬성, 반대예요?" 응시자가 "저는 다소 부정적이었지만, 박 대통령이 시정연설 말씀하셨듯이 어떠한 왜곡이나 미화도 없을 것이라며 객관성, 공정성을 기하겠다고 하였기 때문에 지켜볼 수 밖에 없다" 면서 면접이 끝났다. 이후 응시자는 자신 SNS에 면접 상황을 전하면서 "결국 탈락 소식을 접하였다. 영업관리 직무를 수행하는데 국정 교과서에 대한 견해가 무슨 의미가 있는 것인지 아모레퍼시픽으로부터 탈락 사유에 대한 공식적인 답변을 듣고 싶다"고 밝혔다. 아모레퍼시픽은 "사회에 대한 관심과 답변 내공, 결론 도출 논리성 등을 평가하기 위하여였을뿐, 다른 이유는 없었고, 지원자 성향은 합격 여부에 절대 영향을 주지 않았다. 당사의 채용은 공정성과 투명성을 확보하기 위하여서 개인 정치 성향, 종교, 학연 적절하지 않은 차별을 초래하는 사항들은 묻거나 평가에 반영하지 못하도록 규제하고 있다"고 공식 입장을 밝혔다. 아모레퍼시픽은 공식 사과와 함께 재발방지를 약속하였지만, 대중들은 "기업에서도 사상 검증을 하며 채용을 하는 것이냐"며 비판을 하고 있다.
[10]

《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1]

이사회는 사내이사 3인, 

In [25]:
docs = vectorstore.as_retriever().get_relevant_documents(question)
docs

/tmp/ipykernel_30/2742417880.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = vectorstore.as_retriever().get_relevant_documents(question)


[Document(metadata={'source': 'https://ko.wikipedia.org/wiki/아모레퍼시픽'}, page_content='15년 11월 2일, 아모레퍼시픽이 업계와 면접 당사자에 따르면, 한 면접관은 영업관리직무 2차 면접 시험장에서 "얼마 전 박근혜 대통령님이 국회에서 시정연설을 하며 강한 의지를 표하신 국정교과서에 대하여서 어떤 생각이냐"라는 질문을 하였다. 응시자가 "국정교과서가 사실상 바람직한 결정이라고 할 수 없다" 며 의견을 피력하자, 면접관이 "그래서 국정교과서 찬성, 반대예요?" 응시자가 "저는 다소 부정적이었지만, 박 대통령이 시정연설 말씀하셨듯이 어떠한 왜곡이나 미화도 없을 것이라며 객관성, 공정성을 기하겠다고 하였기 때문에 지켜볼 수 밖에 없다" 면서 면접이 끝났다. 이후 응시자는 자신 SNS에 면접 상황을 전하면서 "결국 탈락 소식을 접하였다. 영업관리 직무를 수행하는데 국정 교과서에 대한 견해가 무슨 의미가 있는 것인지 아모레퍼시픽으로부터 탈락 사유에 대한 공식적인 답변을 듣고 싶다"고 밝혔다. 아모레퍼시픽은 "사회에 대한 관심과 답변 내공, 결론 도출 논리성 등을 평가하기 위하여였을뿐, 다른 이유는 없었고, 지원자 성향은 합격 여부에 절대 영향을 주지 않았다. 당사의 채용은 공정성과 투명성을 확보하기 위하여서 개인 정치 성향, 종교, 학연 적절하지 않은 차별을 초래하는 사항들은 묻거나 평가에 반영하지 못하도록 규제하고 있다"고 공식 입장을 밝혔다. 아모레퍼시픽은 공식 사과와 함께 재발방지를 약속하였지만, 대중들은 "기업에서도 사상 검증을 하며 채용을 하는 것이냐"며 비판을 하고 있다.\n[10]'),
 Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위

In [26]:
docs = vectorstore.similarity_search(question, k=4)
docs

[Document(metadata={'source': 'https://ko.wikipedia.org/wiki/아모레퍼시픽'}, page_content='15년 11월 2일, 아모레퍼시픽이 업계와 면접 당사자에 따르면, 한 면접관은 영업관리직무 2차 면접 시험장에서 "얼마 전 박근혜 대통령님이 국회에서 시정연설을 하며 강한 의지를 표하신 국정교과서에 대하여서 어떤 생각이냐"라는 질문을 하였다. 응시자가 "국정교과서가 사실상 바람직한 결정이라고 할 수 없다" 며 의견을 피력하자, 면접관이 "그래서 국정교과서 찬성, 반대예요?" 응시자가 "저는 다소 부정적이었지만, 박 대통령이 시정연설 말씀하셨듯이 어떠한 왜곡이나 미화도 없을 것이라며 객관성, 공정성을 기하겠다고 하였기 때문에 지켜볼 수 밖에 없다" 면서 면접이 끝났다. 이후 응시자는 자신 SNS에 면접 상황을 전하면서 "결국 탈락 소식을 접하였다. 영업관리 직무를 수행하는데 국정 교과서에 대한 견해가 무슨 의미가 있는 것인지 아모레퍼시픽으로부터 탈락 사유에 대한 공식적인 답변을 듣고 싶다"고 밝혔다. 아모레퍼시픽은 "사회에 대한 관심과 답변 내공, 결론 도출 논리성 등을 평가하기 위하여였을뿐, 다른 이유는 없었고, 지원자 성향은 합격 여부에 절대 영향을 주지 않았다. 당사의 채용은 공정성과 투명성을 확보하기 위하여서 개인 정치 성향, 종교, 학연 적절하지 않은 차별을 초래하는 사항들은 묻거나 평가에 반영하지 못하도록 규제하고 있다"고 공식 입장을 밝혔다. 아모레퍼시픽은 공식 사과와 함께 재발방지를 약속하였지만, 대중들은 "기업에서도 사상 검증을 하며 채용을 하는 것이냐"며 비판을 하고 있다.\n[10]'),
 Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위

In [ ]:
# It seems vectorDB loading from embedding model works fine, but seems llm model does not.
# Some Korean llm model seems to work fine in text-generation task, but for Question-Ansering task, we might need another approach.

## 2. Use QA pipeline with vectorstor similarity search

In [27]:
# import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

# Load model and tokenizer
model_name = "yjgwak/klue-bert-base-finetuned-squard-kor-v1"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set Q_A pipeline
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

config.json:   0%|          | 0.00/635 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/246k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
# Example: define question and context 
question = "오늘 날씨 어때?"
context = "오늘의 날씨는 맑고 따뜻한 기온이 유지될 것으로 보입니다."

# model chain
result = qa_pipeline(question=question, context=context)

# Result
print("질문:", question)
print("답변:", result['answer'])

In [ ]:
# search context in VectorStore
def retrieve_context(question, vectorstore):
    docs = vectorstore.similarity_search(question, k=4)
    if docs:
        return " ".join([doc.page_content for doc in docs])
        # return docs[0].page_content  # return first relevant doc
    else:
        return None

# Generate answer based on query and searched context similar to RAG chain
def answer_question_with_context(question, vectorstore):
    context = retrieve_context(question, vectorstore)
    if context:
        result = qa_pipeline(question=question, context=context)
        return result['answer'], context  # return answer and used source doc
    else:
        return "관련 문맥을 찾지 못했습니다.", None

In [ ]:
# Example
question = "심사위원을 누가 맡았어?"

answer, used_context = answer_question_with_context(question, vectorstore)

print("질문:", question)
print("답변:", answer)
print("사용된 문맥:", used_context)

## 3. Use Gemini+RAG

In [ ]:
# It seems the best and simple and cost-free option when OpenAI api cannot be used.

In [28]:
pip install -q langchain langchain-community langchain_huggingface chromadb google-generativeai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# pip install google-generativeai

In [29]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.llms import OpenAI
import google.generativeai as genai
import os

In [30]:
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = "YOUR-API-KEY"
genai_api_key = "your_API"

In [36]:
genai.configure(api_key=genai_api_key)

In [37]:
# 1. Gemini model
gemini_model = genai.GenerativeModel('gemini-1.5-flash')

# 2. embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [46]:
prompt = f"Question: 집에 가고 싶다 \nAnswer : "
    # response = gemini_model(prompt)
    
response = gemini_model.generate_content(prompt)
answer = response.candidates[0].content.parts[0].text

print(answer)

Several answers are possible depending on the context and desired response. Here are a few options:

* **"알겠어요. 곧 집에 갈 수 있도록 도와드릴게요."** (Algesseoyo. Got jip-e gal su itdorok dowadeurilgesoyo.) - "Okay. I'll help you get home soon."  This is a suitable response if you're in a position to help the person get home.

* **"조금만 더 힘내요! 집에 가면 편히 쉴 수 있어요."** (Jogeumman deo himnaeyo! Jip-e gamyeon pyeonhi swil su isseoyo.) - "Hang in there a little longer! You can rest comfortably when you get home." This is encouraging and offers hope.

* **"집에 가는 길 조심히 가세요."** (Jip-e ganeun gil josimhi gaseyo.) - "Be careful on your way home." This is a polite and caring response.

* **"무슨 일이에요?"** (Museun irieyo?) - "What's wrong?" This is appropriate if you want to understand the reason behind their desire to go home.

* **"언제쯤 집에 갈 수 있어요?"** (Eonjejjeum jip-e gal su isseoyo?) - "When can you go home?" This is a question to clarify their situation.


The best answer depends on the situation.  Consider what k

In [47]:
from langchain.vectorstores import Chroma
# sample docs
docs = [
    Document(page_content="한국어 챗봇은 자연어 처리 기술을 사용하여 사용자와 대화를 나눕니다.", metadata={"source": "doc1"}),
    Document(page_content="인공지능을 활용한 챗봇은 여러 산업에서 사용되고 있습니다.", metadata={"source": "doc2"}),
    Document(page_content="한국어와 영어를 동시에 지원하는 챗봇이 점점 늘어나고 있습니다.", metadata={"source": "doc3"}),
    Document(page_content="챗봇은 고객 서비스를 개선하고 사용자 경험을 향상시키는 데 중요한 역할을 합니다.", metadata={"source": "doc4"})
]

# to avoid collision with previous one
persist_directory = "./new_chroma_db"

vectorstore = Chroma.from_documents(docs, embedding=embedding_model, persist_directory="./chroma_db")

In [51]:
# RAG using prompt
def rag_chatbot(question):
    context_doc = vectorstore.similarity_search(question, k=4)
    # context = context_doc[0].page_content if context_doc else "정보를 찾을 수 없습니다."

    context = " ".join([doc.page_content for doc in context_doc])
    
    prompt = f"Context: {context}\nQuestion: {question}\nAnswer:"
    # response = gemini_model(prompt)
    
    response = gemini_model.generate_content(prompt)
    answer = response.candidates[0].content.parts[0].text

    return answer

In [52]:
# sample question
question = "너는 누구야?"
response = rag_chatbot(question)

print("질문:", question)
print("답변:", response)

질문: 너는 누구야?
답변: 저는 한국어로 응답하는 인공지능 챗봇입니다.  제가 제공하는 정보는 제가 학습한 데이터를 바탕으로 생성됩니다.  위에 제시된 넷플릭스 프로그램 《흑백요리사: 요리 계급 전쟁》에 대한 정보를 포함하여 다양한 주제에 대해 질문에 답변할 수 있습니다.

